## Exploring Sound Generation with Python wave module

Have you ever wondered how a computer is able to play a sound file or how we are able to record and store sound?
How does a sequence of zeroes and ones get converted into physical sound that come out through the speakers?
How are all these real-life analogue sound saved as zeroes and ones?

In this notebook, we will explore the physics of sound and how the information is stored digitally inside a raw WAV file.

1. Writing a WAV file
2. Understanding the data format
3. Physics of sound
4. Representing a note
5. Representing a melody
6. Writing a simple song and saving as a WAV

### 1. Writing a WAV file

In this notebook, we will be using a standard Python module called *wave* that provides basic file writing and reading functionalities. First we will generate a WAV file with 10 seconds of white noise, just to see how the wave module works.

You can also read [this great article](http://soledadpenades.com/2009/10/29/fastest-way-to-generate-wav-files-in-python-using-the-wave-module/) on generating wave files written by [Soledad Penades](https://github.com/sole)

Let's start by importing the *wave* module

In [1]:
import wave

We will also import:
* *random* module for generating random bits for the random noise
* *struct* module for manipulating bytes

In [2]:
import random, struct

Now we can use the wave module to create an instance of the file that we will be writing.

In [3]:
wav_file = wave.open('output/myNoise.wav', 'w')

According to the [documentation](https://hg.python.org/cpython/file/3.5/Lib/wave.py) for the wave module, "You should set the parameters before the first writeframesraw or writeframes."

To set the parameters, we will call the [setparams((nchannels, sampwidth, framerate, nframes, comptype, compname))](https://docs.python.org/3/library/wave.html#wave.Wave_write.setparams) method on the *wav_file* instance.

The parameters will be:
* nchannels (number of audio channels) : 2    #for stereo
* sampwidth (number of bytes per audio sample) : 2
* framerate (number of samples per second) : 44100
* nframes (the number of audio frames written to the header) : 0    #seekable output stream
* comptype (compression type) : 'NONE'                              #raw
* compname (human-readable compression type) : 'not compressed'     #raw

We will be reusing the value for framerate, so we can assign a name to it.

In [4]:
FRAMERATE = 44100
wav_file.setparams((2, 2, FRAMERATE, 0, 'NONE', 'not compressed'))

Next we will create the bytes representing the white noise.

In [5]:
SOUND_LENGTH = 10
bits = []
for i in range(0, round(FRAMERATE * SOUND_LENGTH)):
    bit = random.randint(-20000, 20000)
    packed_bit = struct.pack('h', round(bit))
    bits.append(packed_bit)
    bits.append(packed_bit)
bits = (b''.join(bits))

Finally, we write these bytes into the file, and close the file to complete the process.

In [6]:
wav_file.writeframes(bits)
wav_file.close()

The following is the entire script for this section:
```
import wave, random, struct

FRAMERATE = 44100
SOUND_LENGTH = 10

wav_file = wave.open('output/myNoise.wav', 'w')
wav_file.setparams((2, 2, FRAMERATE, 0, 'NONE', 'not compressed'))

bits = []
for i in range(0, round(FRAMERATE * SOUND_LENGTH)):
    bit = random.randint(-20000, 20000)
    packed_bit = struct.pack('h', round(bit))
    bits.append(packed_bit)
    bits.append(packed_bit)
bits = (b''.join(bits))
wav_file.writeframes(bits)
wav_file.close()
```

### 2. Understanding the data format



### 3. Physics of sound

### 4. Representing a note

For generating white noise, we selected a random integer between -20000 and 20000 at each frame.
To generate a note at a certain pitch, we need to store the soundwave's value at each frame.

In [7]:
#Python 3
import wave, random, struct, math

FRAMERATE = 44100
SOUND_LENGTH = 5

wav_file = wave.open('output/myNote.wav', 'w')
wav_file.setparams((2, 2, FRAMERATE, 0, 'NONE', 'not compressed'))

Middle A has a frequency of 440 hz, so the "wavelength" in frames is equal to 1 sec * 44100 frames/sec / 440.

In [8]:
WAVELENGTH = FRAMERATE / 440
bits = []
for i in range(0, round(FRAMERATE * SOUND_LENGTH)):
    bit = 5000 * math.sin( i * (2 * math.pi / WAVELENGTH) )
    packed_bit = struct.pack('h', round(bit))
    bits.append(packed_bit)
    bits.append(packed_bit)
bits = (b''.join(bits))
wav_file.writeframes(bits)
wav_file.close()

```
#Python 3
import wave, random, struct, math

FRAMERATE = 44100
SOUND_LENGTH = 5
WAVELENGTH = FRAMERATE / 440

wav_file = wave.open('output/myNote.wav', 'w')
wav_file.setparams((2, 2, FRAMERATE, 0, 'NONE', 'not compressed'))

bits = []
for i in range(0, round(FRAMERATE * SOUND_LENGTH)):
    bit = 5000 * math.sin( i * (2 * math.pi / WAVELENGTH) )
    packed_bit = struct.pack('h', round(bit))
    bits.append(packed_bit)
    bits.append(packed_bit)
bits = (b''.join(bits))
wav_file.writeframes(bits)
wav_file.close()
```

### 5. Representing a melody

In [9]:
class Note():
    def __init__(self, frequency=440, length=1):
        self.frequency = frequency
        self.length = length
        self.wavelength = FRAMERATE / self.frequency
    
    def toBytes(self):
        bits = []
        for i in range(0, round(self.length * FRAMERATE)):
            bit = 5000 * math.sin( i * (2 * math.pi / self.wavelength) )
            packed_bit = struct.pack('h', round(bit))
            bits.append(packed_bit)
            bits.append(packed_bit)
        bits = (b''.join(bits))
        return bits

In [10]:
class Sequence():
    def __init__(self):
        self.notes = []
    
    def add(self, note):
        if type(note) == Note:
            self.notes.append(note)
        else:
            raise TypeError('Should be a Note object')
    
    def addNote(self, *args, **kwargs):
        self.notes.append(Note(*args, **kwargs))
    
    def writeWav(self, filename):
        wav_file = wave.open(filename, 'w')
        wav_file.setparams((2, 2, FRAMERATE, 0, 'NONE', 'not compressed'))
        bits = b''
        for note in self.notes:
            bits += note.toBytes()
        wav_file.writeframes(bits)
        wav_file.close()

In [11]:
doremi = Sequence()
doremi.addNote(261.626, 1)
doremi.addNote(293.665, 1)
doremi.addNote(329.628, 1)
doremi.writeWav('output/myMelody.wav')

### 6. Writing a simple song and saving as a WAV

In [12]:
import sound

BPM = 120
b = 60 / BPM

twinkle = sound.Sequence()
twinkle.addNote('C4', b)
twinkle.addNote('C4', b)
twinkle.addNote('G4', b)
twinkle.addNote('G4', b)
twinkle.addNote('A5', b)
twinkle.addNote('A5', b)
twinkle.addNote('G4', b * 2)
twinkle.addNote('F4', b)
twinkle.addNote('F4', b)
twinkle.addNote('E4', b)
twinkle.addNote('E4', b)
twinkle.addNote('D4', b)
twinkle.addNote('D4', b)
twinkle.addNote('C4', b * 2)
twinkle.writeWav('output/twinkle')

In [13]:
import nb_embed
nb_embed.html('''
<table>
    <tr><td>1</td><td>0</td></tr>
    <tr><td>0</td><td>1</td></tr>
</table>
''')

1,0
0,1
